In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Window-functions').master('local[1]').getOrCreate()

In [7]:
emp = [(1,"Smith",-1,"2018","10","M",3000),(8,"SmithV",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]

empColumns = ["emp_id","name","superior_emp_id","year_joined","emp_dept_id","gender","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show(truncate=False)

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|8     |SmithV  |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1   

In [8]:
empDF.createOrReplaceTempView("employee")
spark.sql('show tables').show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        | employee|       true|
+--------+---------+-----------+



In [4]:
spark.sql('select * from employee').show()

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|
|     2|    Rose|              1|       2010|         20|     M|  4000|
|     3|Williams|              1|       2010|         10|     M|  1000|
|     4|   Jones|              2|       2005|         10|     F|  2000|
|     5|   Brown|              2|       2010|         40|      |    -1|
|     6|   Brown|              2|       2010|         50|      |    -1|
+------+--------+---------------+-----------+-----------+------+------+



In [5]:
'''Want to get the result salary in desc order for every department'''

q = ' select * from employee order by emp_dept_id asc, salary desc'

spark.sql(q).show()

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|
|     4|   Jones|              2|       2005|         10|     F|  2000|
|     3|Williams|              1|       2010|         10|     M|  1000|
|     2|    Rose|              1|       2010|         20|     M|  4000|
|     5|   Brown|              2|       2010|         40|      |    -1|
|     6|   Brown|              2|       2010|         50|      |    -1|
+------+--------+---------------+-----------+-----------+------+------+



In [ ]:
'''Window Ranking Functions'''

1.Row_Number() - will give you sequence of row_numbers in each group 

2.Rank() - will give you seq of ranks in each group but can skip ranks if finds any duplicate values

3.Dense_Rank() - will give you seq of ranks in each group but can't skip ranks if finds any duplicate values


In [9]:
q = 'select *,row_number() over(partition by emp_dept_id order by salary desc) as rnk from employee'

spark.sql(q).show()

+------+--------+---------------+-----------+-----------+------+------+---+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|rnk|
+------+--------+---------------+-----------+-----------+------+------+---+
|     5|   Brown|              2|       2010|         40|      |    -1|  1|
|     2|    Rose|              1|       2010|         20|     M|  4000|  1|
|     1|   Smith|             -1|       2018|         10|     M|  3000|  1|
|     8|  SmithV|             -1|       2018|         10|     M|  3000|  2|
|     4|   Jones|              2|       2005|         10|     F|  2000|  3|
|     3|Williams|              1|       2010|         10|     M|  1000|  4|
|     6|   Brown|              2|       2010|         50|      |    -1|  1|
+------+--------+---------------+-----------+-----------+------+------+---+



In [11]:
q1 = 'select *,rank() over(partition by emp_dept_id order by salary desc) as rnk from employee'

spark.sql(q1).show()

+------+--------+---------------+-----------+-----------+------+------+---+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|rnk|
+------+--------+---------------+-----------+-----------+------+------+---+
|     5|   Brown|              2|       2010|         40|      |    -1|  1|
|     2|    Rose|              1|       2010|         20|     M|  4000|  1|
|     1|   Smith|             -1|       2018|         10|     M|  3000|  1|
|     8|  SmithV|             -1|       2018|         10|     M|  3000|  1|
|     4|   Jones|              2|       2005|         10|     F|  2000|  3|
|     3|Williams|              1|       2010|         10|     M|  1000|  4|
|     6|   Brown|              2|       2010|         50|      |    -1|  1|
+------+--------+---------------+-----------+-----------+------+------+---+



In [12]:
q2 = 'select *,dense_rank() over(partition by emp_dept_id order by salary desc) as rnk from employee'

spark.sql(q2).show()

+------+--------+---------------+-----------+-----------+------+------+---+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|rnk|
+------+--------+---------------+-----------+-----------+------+------+---+
|     5|   Brown|              2|       2010|         40|      |    -1|  1|
|     2|    Rose|              1|       2010|         20|     M|  4000|  1|
|     1|   Smith|             -1|       2018|         10|     M|  3000|  1|
|     8|  SmithV|             -1|       2018|         10|     M|  3000|  1|
|     4|   Jones|              2|       2005|         10|     F|  2000|  2|
|     3|Williams|              1|       2010|         10|     M|  1000|  3|
|     6|   Brown|              2|       2010|         50|      |    -1|  1|
+------+--------+---------------+-----------+-----------+------+------+---+



In [13]:
'''get the second highest salary in each department'''

q3 = '''select * from (select *,dense_rank() over(partition by emp_dept_id order by salary desc) as rnk from employee) a 
      where rnk=2'''

spark.sql(q3).show()

+------+-----+---------------+-----------+-----------+------+------+---+
|emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|rnk|
+------+-----+---------------+-----------+-----------+------+------+---+
|     4|Jones|              2|       2005|         10|     F|  2000|  2|
+------+-----+---------------+-----------+-----------+------+------+---+

